# <a id='toc1_'></a>[TWO MOONS DATASET EXPERIMENTS](#toc0_)

The purpose of this notebook will be to analyze the behavior of the networks on a toy task, namely: the two moons dataset. This dataset is a simple binary classification task, where the goal is to separate two half-moon shaped clusters. The dataset is generated using the `make_moons` function from `sklearn.datasets`.

We want to see if the network manages to have uncertainty in the regions where the two moons are close to each other, and if it is able to correctly classify the points in the two moons.

**Table of contents**<a id='toc0_'></a>    
- [TWO MOONS DATASET EXPERIMENTS](#toc1_)    
  - [Imports](#toc1_1_)    
  - [Train / test split](#toc1_2_)    
  - [Configuration of the network](#toc1_3_)    
  - [Training the network](#toc1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Imports](#toc0_)
Importing libraries and setting the global variables

In [ ]:
from utils import *
from dataloader import *
import models
import torch
import trainer
from optimizer import *
from sklearn.datasets import make_moons

SEED = 2506  # Random seed
N_NETWORKS = 1  # Number of networks to train
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### PATHS ###
SAVE_FOLDER = "saved-two-moons"
DATASETS_PATH = "datasets"

### SEED ###
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.set_default_device(DEVICE)
    torch.set_default_dtype(torch.float32)

: 

## <a id='toc1_2_'></a>[Train / test split](#toc0_)

We split the dataset into a train and test set. The test set will be used to evaluate the performance of the network, while the train set will be used to train the network. We separate the data into:
- `x_train`: the input data for the training set with 1024 points
- `y_train`: the labels for the training set with 1024 points
- `x_test`: the input data for the test set with 256 points
- `y_test`: the labels for the test set with 256 points

In [ ]:
### CREATE TWO MOONS DATASET ###
X, y = make_moons(n_samples=1280, noise=0.1, random_state=SEED)

# normalize dataset
X = (X - X.mean(axis=0)) / X.std(axis=0)

X_train = X[:1024]
y_train = y[:1024]
X_test = X[1024:]
y_test = y[1024:]

train_tensor = GPUTensorDataset(
    torch.from_numpy(X_train).float(), torch.from_numpy(y_train))
test_tensor = GPUTensorDataset(
    torch.from_numpy(X_test).float(), torch.from_numpy(y_test))
train_loader = GPUDataLoader(train_tensor, batch_size=128, shuffle=True)
test_loader = GPUDataLoader(test_tensor, batch_size=1280-1024)
INPUT_SIZE = train_tensor.data.shape[1]

: 

In [ ]:
### PLOT X, y ###
# subplot for training data and test data
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].grid(True)
ax[1].grid(True)
ax[0].scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.Spectral, edgecolors='k', zorder=2)
ax[0].set_title("Training data")
ax[1].scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.Spectral, edgecolors='k', zorder=2)
ax[1].set_title("Test data")
plt.show()

: 

## <a id='toc1_3_'></a>[Configuration of the network](#toc0_)

We configure the network by setting the parameters within a dictionary.

In [ ]:

### NETWORK CONFIGURATION ###
networks_data = [
    {
        "nn_type": models.BNN,
        "nn_parameters": {
            "layers": [INPUT_SIZE, 2048, 2048, 10],
            "init": "uniform",
            "device": DEVICE,
            "std": 0.1,
            "dropout": False,
            "batchnorm": True,
            "bnmomentum": 0.15,
            "bneps": 1e-5,
            "latent_weights": False,
            "running_stats": False,
        },
        "training_parameters": {
            'n_epochs': 100,
            'batch_size': 128,
            'test_mcmc_samples': 1,
        },
        "criterion": torch.functional.F.nll_loss,
        "reduction": "mean",
        "optimizer": BinarySynapticUncertainty,
        "optimizer_parameters": {
            "temperature": 1,
            "num_mcmc_samples": 1,
            "init_lambda": 0,
            "lr": 1,
            "metaplasticity": 0.35,
            "gamma": 0,
        },
    }
]

: 

## <a id='toc1_4_'></a>[Training the network](#toc0_)

In [ ]:
for index, data in enumerate(networks_data):

    ### ACCURACY INITIALIZATION ###
    accuracies = []
    batch_size = data['training_parameters']['batch_size']
    padding = data['padding'] if 'padding' in data else 0
    ### FOR EACH NETWORK IN THE DICT ###
    for iteration in range(N_NETWORKS):
        ### SEED ###
        torch.manual_seed(SEED + iteration)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(SEED + iteration)

        ### NETWORK INITIALIZATION ###
        model = data['nn_type'](**data['nn_parameters'])

        ### INSTANTIATE THE TRAINER ###
        if data["optimizer"] in [BinarySynapticUncertainty, BayesBiNN]:
            network = trainer.BayesTrainer(batch_size=batch_size,
                                            model=model, **data, device=DEVICE)
        else:
            network = trainer.GPUTrainer(batch_size=batch_size,
                                            model=model, **data, device=DEVICE, logarithmic=True)

        ### TRAINING ###

        network.fit(
            train_loader, **data['training_parameters'], test_loader=[test_loader], verbose=True)


: 

## Exploiting the results

In [ ]:
# Use the predictions of the network to plot the uncertainty of the measures
N_TEST_SAMPLES = 1000
predictions = []
for inputs, targets in test_loader:
    # gives us the mean and the std of the predictions
    predictions.append(network.predict(inputs.to(DEVICE), N_TEST_SAMPLES))

mean = torch.cat([p[0] for p in predictions], dim=0)
std = torch.cat([p[1] for p in predictions], dim=0)

: 